# Historical Data: Intraday

These financial data are generally provided in a format that includes the following information:  
Date  
Open Price  
High Price  
Low Price  
Closing Price  
Volume
These data—often referred to as OHLC Chart Data—can be interpreted as Time Series data and are perfect for performing technical analysis. We’ll dive into this format in just a moment but, for now, just realize this is a standard format for historical pricing data within financial markets.

## [yfinance]()

For this approach, we need to install the yfinance library as `pip install yfinance`. This library provides sample tools for working with financial data requests to the Yahoo Finance website.  
> ⚠️ periodic breakage, general quirkiness

In [ ]:
import yfinance as yf
import pandas as pd
tick = ['NVDA']
#for i in tick:
data = yf.download('NVDA', period='max', auto_adjust=True)
print(data.index)

In [ ]:
pip install yahoofinance

In [ ]:
from yahoofinance import HistoricalPrices
req = HistoricalPrices('AAPL', '1999-01-22', '2022-09-15')

# Quandl:
Quandl offers official APIs to access any public dataset for free. One important note is that the **free Quandl OHLC data only goes up to 2018**.  

 ⚠️ Providing Only US market Data.


In [ ]:
import quandl
#import time
#tick = ['NVDA','MSFT','AAC','AACG', 'AACI','AACIW','AADI','AAIC','AAL','AAMC','AAME','AAN','AAOI','AAON','AAP','AAPL','AAQC','AAT','AATC','AAU','AAWW','AB','ABB','ABBV','ABC']
# Get data via Quandl API
#data = quandl.get('WIKI/NVDA')
#data.info()
msft = quandl.get('WIKI/MSFT', start_date='1986-03-13', end_date='15-09-2022')
msft.info()

# Alpha Vantage:
 ✌️ Providing Free 20 years of data with no missing values.  
 ⚠️ period breakage

In [ ]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&outputsize=full&apikey=UNO4CZQHSBZSN71N'
r = requests.get(url)
data= r.json()
timeseries = data['Time Series (Daily)']
value2 = dict()
extra = list()
index = ['open', 'high', 'low', 'close', 'volume']
for value in timeseries:
    for key in timeseries.get(value):
        extra.append(timeseries[value][key])
    value2[value]=extra
    extra = list()

    
df = pd.DataFrame(value2, index=index).T
print(df.head())


## Zerodha-historical-data-download :
This contains code to download historical data for more than 2000 days and intraday data for more than 100 days.  
⚠️ Not provinding free historical data


In [ ]:
from kiteconnect import KiteConnect, KiteTicker
import pandas as pd
import datetime as dt
import numpy as np
import os
import time
import json



api_key = 'your_api_key'
api_secret = 'your_api_secret'

kite = KiteConnect(api_key)
url = kite.login_url()

request_token = 'your_request_token'
session = kite.generate_session(request_token, api_secret)
access_token = session['access_token']
kite.set_access_token(access_token)

instruments = pd.DataFrame(kite.instruments('NSE'))


def lookup(df, symbol):
    try:
        return df[df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

def fetchandstoreOHLC(symbol,duration,tf):
    nodata=[]
    try:
        if not os.path.isfile(symbol+'.csv'):
            data = pd.DataFrame()
            instrument_token = lookup(instruments, symbol)  
            if duration>100:
                no_of_hundreds = duration//100
                start = 100
                end = 0
                for i in range(no_of_hundreds):
                    df = pd.DataFrame(kite.historical_data(instrument_token, from_date = dt.datetime.today()-dt.timedelta(start), 
                    to_date = dt.datetime.today()-dt.timedelta(end), interval = tf))
                    data = data.append(df)
                    start+=100
                    end+=100
                dur1 = duration%100
                df1 = pd.DataFrame(kite.historical_data(instrument_token, from_date = dt.datetime.today()-dt.timedelta(duration), 
                                      to_date = dt.datetime.today()-dt.timedelta(duration-dur1), interval = tf))
                data = data.append(df1)
            else:
                df2 = pd.DataFrame(kite.historical_data(instrument_token, from_date = dt.datetime.today()-dt.timedelta(duration), 
                                      to_date = dt.datetime.today(), interval = tf))
                data = data.append(df2)
            data.set_index("date", inplace = True)
            data.sort_index(ascending= True,inplace = True)
            print('Storing {} days data for {}'.format(duration, symbol))
            data.to_csv(symbol+'.csv')
        else:
            print('Data already available for{}'.format(symbol))
            return
    except:
        print("skipping for {}".format(symbol))
        nodata.append(symbol)
    
tickers = ['HDFCBANK','ICICIBANK','KOTAKBANK', 'AXISBANK', 'SBIN', 'RELIANCE','TCS','INFY','HINDUNILVR','HDFC','BAJFINANCE','WIPRO','BHARTIARTL','HCLTECH','ASIANPAINT','ITC','LT','ULTRACEMCO',
           'MARUTI','SUNPHARMA','TATASTEEL','JSWSTEEL','TITAN','ADANIPORTS','ONGC','HDFCLIFE','TECHM','DIVISLAB','POWERGRID','SBILIFE','NTPC','BAJAJ-AUTO','BPCL','IOC','M&M','SHREECEM','HINDALCO',
           'GRASIM','BRITANNIA','TATAMOTORS','COALINDIA','TATACONSUM','INDUSINDBK','DRREDDY','CIPLA','EICHERMOT','UPL','NESTLEIND','HEROMOTOCO','NIFTY 50','NIFTY BANK']


for ticker in tickers:
    fetchandstoreOHLC(ticker, 1095, '5minute')


### [Angel One]():
SMARTAPI-PYTHON is a Python library for dealing AMX,that is a set of REST-like HTTP APIs that expose many capabilities required to build stock market investment and trading platform.
API is as below:  
https://apiconnect.angelbroking.com/rest/secure/angelbroking/historical/v1/getCandleData
 
> NOTE:  Historical API only available for NSE Equity(NSE) and NSE Futures (NFO) Segment  


In [ ]:
from smartapi.smartConnect import SmartConnect
import pandas as pd
import requests
feed_token = None
token_map = None

obj = SmartConnect(api_key="TkDi9GY2")
data = obj.generateSession("R12345", "Tannu@123")
print(data)
refreshToken=data['data']['refreshToken']

#fetch the feedtoken
feedToken = obj.getfeedToken()
feed_token = feedToken
#fetch User Profile
userProfile = obj.getProfile((refreshToken))
print(userProfile)

#historic api
def OHLCHistory(symbol, token, interval, fdate, todate):
    try:
        historicParam={
            "exchange":"NSE",
            "tradingsymbol":symbol,
            "symboltoken":token,
            "interval":interval,
            "fromdate":fdate,
            "todate":todate
        }
        history = obj.getCandleData(historicParam)['data']
        history = pd.DataFrame(history)

        history = history.rename(
            columns={0: "DateTime", 1: "open", 2: "high", 3: "low", 4: "close", 5:"Volumne"}
        )
        history['DateTime']=pd.to_datetime(history['DataTime'])
        history=history.set_index('DateTime')
        return history
    except Exception as e:
        print("Histpric api failed: {}".format(e))
data = OHLCHistory("SBIN-EQ", "3045", "ONE_DAY", "2020-02-08 00:00", "2021-02-08 15:30")
data_df = pd.DataFrame(data)
data_df.to_csv(r'{}.csv'.format("SBIN_EQ"), index=True, header=False)

### [Polygio.io]()

#### Parameters

|Parameter|Default value|Explanation|
|---|---|---|
|multiplier |1 |The size of the timespan multiplier.|
|timespan|day|The size of the time window.|
|limit|120|Limits the number of base aggregates queried to create the aggregate results. Max 50000 and Default 5000.|

In [ ]:
from sqlite3 import adapters
from polygon import RESTClient
from local_settings import polygon as settings
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import datetime

markets = ['crypto', 'stocks']

class MyRESTClient(RESTClient):
    def __init__(self, auth_key: str, timeout: int=None):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10, backoff_factor=10, status_forcelist=[429, 500, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self._session.mount('https://', adapter)
    
    def get_bars(self, market:str=None, ticker:str=None, multiplier:int=1, timespan:str='minute', from_:datetime=None, to:datetime=None) -> pd.DataFrame:
        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')
        
        if ticker is None:
            raise Exception('Ticker must not be None.')
        
        from_ = from_ if from_ else data(2000, 1, 1)
        to = to if to else date.today()

        if market == 'stocks':
            resp = self.stocks_aggregates(ticker, multiplier, timespan, from_.strftime('%Y-%m-%d'), to.strftime('%Y-%m-%d'), limit=50000)
            df = pd.DataFrame(resp.results)
            last_minute = 0
            while resp.results[-1]['t'] > last_minute:
                last_minute = resp.results[-1]['t']
                last_minute_date = datetime.fromtimestamp(last_minute/1000).strftime('%Y-%m-%d')
                resp = self.stocks_aggregates(ticker, multiplier, timespan, from_.strftime('%Y-%m-%d'), to.strftime('%Y-%m-%d'), limit=50000)
                new_bars = pd.DataFrame(resp.results)
                df = df.append(new_bars[new_bars['t']>last_minute])

            df['date'] = pd.to_datetime(df['t'], unit='ms')
            df = df.rename(columns={
                'o':'open',
                'h':'high',
                'l':"low",
                'c':'close',
                'v':'volume',
                'vw':'vwap',
                'n':'transactions'})
            
            df = df[['date', 'open', 'high', 'low' ,'close', 'volume']]
            return df
        return None


client = MyRESTClient(polygon['api_key'])
ticker='AAPL'
df = client.get_bars(market="stocks", ticker=ticker)
df.to_csv(r'{}.csv'.format(ticker))

### [Kotak Securities]()

All URIs are relative to https://tradeapi.kotaksecurities.com/apim



In [ ]:
from ks_api_client import ks_api

client = ks_api.KSTradeApi(access_token = "access_token", userid = "userid", \
                 consumer_key = "consumer_key", ip = "IP", app_id = "app_id")

#First initialize session and generate session token

try:
    # Get historical prices
    client.history("historicalprices",{"exchange":"bse","cocode":"476","fromdate":"01-jan-2014","todate":"08-oct-2015"})
except Exception as e:
    print("Exception when calling Historical API->details: %s\n" % e)

### [ALice Blue API]()

1. Historical data API will be available from 5:30 PM (evening) to 8 AM (Next day morning) on weekdays (Monday to Friday). Historical data will not be available during market hours.  
2. Historical data API will be available fully during weekends and holidays.  
3. For NSE segment, 2 years of historical data will be provided.  
4. For NFO, CDS and MCX segments, current expiry data will be provided.  

In [ ]:
from alice_blue import *
session_id = AliceBlue.login_and_get_sessioID(username = "username", password = "password", twoFA= "1993", app_id = "app_id", api_secret = "api_secret")
data = pd.DataFrame(alice.historical_data(alice.get_instrument_by_symbol("NSE"), datetime.datetime.now(), datetime.datetime.now(), HistoricalDataType.Minute))
data.to_csv("NSE.csv", header=False)